# Практична робота №4 | Спеліна Віталій | ОІ-21сп | Варіант 4

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

In [20]:
import warnings
warnings.simplefilter('ignore')

In [21]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [22]:
data = pd.read_csv('./new_data.csv')

# Налаштування моделі

In [23]:
X = data.drop(columns=['Unnamed: 0', 'Target_Graduate', 'Target_Enrolled'])
y = np.where((data['Target_Graduate'] == 1) | (data['Target_Enrolled'] == 1), 0, 1)  # 0 - не відрахований, 1 - відрахований
X_columns = X.columns.tolist()

old_class_counts = pd.Series(y).value_counts()
print(old_class_counts)

0    3003
1    1421
Name: count, dtype: int64


# Балансування класів

In [24]:
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

y = np.select(
    [data['Target_Graduate'] == 1, data['Target_Enrolled'] == 1],
    [0, 1],  # 0 - Graduate, 1 - Enrolled
    default=2  # 2 - Dropout, якщо обидва поля рівні 0
)

old_class_counts = pd.Series(y).value_counts()
print("Кількість зразків у початкових класах:\n", old_class_counts)

X = data.drop(columns=['Unnamed: 0', 'Target_Graduate', 'Target_Enrolled'])
X_columns = X.columns.tolist()

smote = SMOTE(sampling_strategy='auto', random_state=42)
X, y = smote.fit_resample(X, y)

new_class_counts = pd.Series(y).value_counts()
print("Кількість зразків у збалансованих класах:\n", new_class_counts)

X = data.drop(columns=['Unnamed: 0', 'Target_Graduate', 'Target_Enrolled'])
y = np.where((data['Target_Graduate'] == 1) | (data['Target_Enrolled'] == 1), 0, 1)  # 0 - не відрахований, 1 - відрахований

Кількість зразків у початкових класах:
 0    2209
2    1421
1     794
Name: count, dtype: int64
Кількість зразків у збалансованих класах:
 2    2209
0    2209
1    2209
Name: count, dtype: int64


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8)

In [26]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3539, 97)
(3539,)
(885, 97)
(885,)


## На минулій практичній роботі було визначено три найкращі моделі для мого завдання: 
### - Random Forest
### - Gradient Boosting
### - Logistic Regression

# Результати моделей з параметрами за замовчуванням

In [27]:
# Random Forest
rf_default = RandomForestClassifier(random_state=42)
rf_default.fit(X_train, y_train)

# Gradient Boosting
gb_default = GradientBoostingClassifier(random_state=42)
gb_default.fit(X_train, y_train)

# Logistic Regression
lr_default = LogisticRegression(random_state=42, max_iter=500)
lr_default.fit(X_train, y_train)

models_default = {
    'Random Forest': rf_default,
    'Gradient Boosting': gb_default,
    'Logistic Regression': lr_default
}

for model_name, model in models_default.items():
    print(f"--- {model_name} ---")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

--- Random Forest ---
              precision    recall  f1-score   support

           0       0.87      0.93      0.90       605
           1       0.83      0.70      0.76       280

    accuracy                           0.86       885
   macro avg       0.85      0.82      0.83       885
weighted avg       0.86      0.86      0.86       885

--- Gradient Boosting ---
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       605
           1       0.83      0.74      0.78       280

    accuracy                           0.87       885
   macro avg       0.86      0.84      0.84       885
weighted avg       0.87      0.87      0.87       885

--- Logistic Regression ---
              precision    recall  f1-score   support

           0       0.89      0.91      0.90       605
           1       0.80      0.75      0.77       280

    accuracy                           0.86       885
   macro avg       0.84      0.83      0.84       88

## Визначення сітки гіперпараметрів для кожної з моделей

In [28]:
# Гіперпараметри для RandomForest
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

# Гіперпараметри для GradientBoosting
gb_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0]
}

# Гіперпараметри для Logistic Regression
lr_params = {
    'C': [0.01, 0.1, 1],
    'penalty': ['l2'],
    'class_weight': ['balanced', None]
}


# Оптимізація гіперпараметрів для кожної моделі за допомогою GridSearchCV

In [29]:
# RandomForest
rf = RandomForestClassifier(random_state=42)
rf_grid_search = GridSearchCV(rf, rf_params, scoring='recall', cv=5, n_jobs=-1, return_train_score=True)
rf_grid_search.fit(X_train, y_train)
rf_best_model = rf_grid_search.best_estimator_
print("The best parameters are %s with a score of %0.2f"
      % (rf_grid_search.best_params_, rf_grid_search.best_score_))
grid_results = pd.concat([pd.DataFrame(rf_grid_search.cv_results_["params"]),
                          pd.DataFrame(rf_grid_search.cv_results_["mean_test_score"], 
                          columns=["recall"])],
                          axis=1)

grid_results

The best parameters are {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100} with a score of 0.79


,class_weight,max_depth,min_samples_split,n_estimators,recall
0,balanced,5,2,100,0.794032
1,balanced,5,2,200,0.791400
2,balanced,5,5,100,0.794898
3,balanced,5,5,200,0.794028
4,balanced,10,2,100,0.759833
5,balanced,10,2,200,0.755447
6,balanced,10,5,100,0.765096
7,balanced,10,5,200,0.763338
8,balanced,20,2,100,0.700253
9,balanced,20,2,200,0.705501


In [30]:
# Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)
gb_grid_search = GridSearchCV(gb, gb_params, scoring='recall', cv=5, n_jobs=-1, return_train_score=True)
gb_grid_search.fit(X_train, y_train)
gb_best_model = gb_grid_search.best_estimator_
print("The best parameters are %s with a score of %0.2f"
      % (gb_grid_search.best_params_, gb_grid_search.best_score_))
grid_results = pd.concat([pd.DataFrame(gb_grid_search.cv_results_["params"]),
                          pd.DataFrame(gb_grid_search.cv_results_["mean_test_score"], 
                          columns=["recall"])],
                          axis=1)

grid_results

The best parameters are {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0} with a score of 0.74


,learning_rate,max_depth,n_estimators,subsample,recall
0,0.01,3,100,0.8,0.685352
1,0.01,3,100,1.0,0.680974
2,0.01,3,200,0.8,0.705501
3,0.01,3,200,1.0,0.700249
4,0.01,5,100,0.8,0.671325
5,0.01,5,100,1.0,0.672209
6,0.01,5,200,0.8,0.703754
7,0.01,5,200,1.0,0.702881
8,0.10,3,100,0.8,0.744070
9,0.10,3,100,1.0,0.733556


In [31]:
# Logistic Regression
lr = LogisticRegression(random_state=42, max_iter=500)
lr_grid_search = GridSearchCV(lr, lr_params, scoring='recall', cv=5, n_jobs=-1, return_train_score=True)
lr_grid_search.fit(X_train, y_train)
lr_best_model = lr_grid_search.best_estimator_
print("The best parameters are %s with a score of %0.2f"
      % (lr_grid_search.best_params_, lr_grid_search.best_score_))
grid_results = pd.concat([pd.DataFrame(lr_grid_search.cv_results_["params"]),
                          pd.DataFrame(lr_grid_search.cv_results_["mean_test_score"], 
                          columns=["recall"])],
                          axis=1)

grid_results

The best parameters are {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2'} with a score of 0.84


,C,class_weight,penalty,recall
0,0.01,balanced,l2,0.822068
1,0.01,None,l2,0.731793
2,0.10,balanced,l2,0.835206
3,0.10,None,l2,0.763342
4,1.00,balanced,l2,0.831709
5,1.00,None,l2,0.754562


# Оцінка метрик для кожної моделі на тестовому наборі

In [32]:
models = {
    'Random Forest': rf_best_model,
    'Gradient Boosting': gb_best_model,
    'Logistic Regression': lr_best_model
}

for model_name, model in models.items():
    print(f"--- {model_name} ---")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

--- Random Forest ---
              precision    recall  f1-score   support

           0       0.89      0.89      0.89       605
           1       0.76      0.78      0.77       280

    accuracy                           0.85       885
   macro avg       0.83      0.83      0.83       885
weighted avg       0.85      0.85      0.85       885

--- Gradient Boosting ---
              precision    recall  f1-score   support

           0       0.89      0.94      0.91       605
           1       0.84      0.74      0.79       280

    accuracy                           0.87       885
   macro avg       0.86      0.84      0.85       885
weighted avg       0.87      0.87      0.87       885

--- Logistic Regression ---
              precision    recall  f1-score   support

           0       0.91      0.86      0.89       605
           1       0.74      0.82      0.77       280

    accuracy                           0.85       885
   macro avg       0.82      0.84      0.83       88

# Висновок

### Було проведення балансування класів за допомогою SMOTE.

### Було проведено налаштування гіперпараметрів за методом Grid Search, та результати:

Gradient Boosting до і після налаштування показав найбільш стабільні результати з найвищою точністю, високими показниками f1-score і балансом precision та recall для обох класів, особливо для менш чисельного класу 1.
Logistic Regression після налаштування покращив метрики для класу 1, що може бути корисним, якщо пріоритетом є розпізнавання рідкісного класу (Dropout).
Random Forest показав деяке погіршення метрик після налаштування, що вказує на

Загалом, Gradient Boosting залишається найефективнішою моделлю для поточного завдання, однак Logistic Regression також може бути корисною альтернативою для покращення прогнозування класу відрахованих.